# Day6_3: 통합 실습 - 다중 소스 데이터 분석 (정답)

## 📚 학습 목표

**Part 1: 기초**
1. CSV 파일과 SQLite 데이터베이스에서 데이터 로드하기
2. pandas merge를 사용한 데이터 통합 (inner/left/right/outer join)
3. groupby를 활용한 다차원 집계 분석
4. 집계 함수 조합 (sum, mean, count, max, min)
5. 결측치 처리 및 데이터 품질 검증

**Part 2: 심화**
1. Plotly Graph Objects로 복합 차트 생성
2. 서브플롯을 활용한 통합 대시보드 구성
3. 인터랙티브 기능 추가 (hover, 드릴다운)
4. HTML 리포트 자동 생성 및 배포

---

## 🎯 실습 퀴즈 정답

---

### 사전 준비: 데이터 및 라이브러리 로드

In [ ]:
import pandas as pd
import sqlite3
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from datetime import datetime, timedelta

# 데이터 생성 (문제 노트북과 동일)
np.random.seed(42)
start_date = datetime(2025, 10, 1)
dates = [start_date + timedelta(days=x) for x in range(90)]

orders_data = {
    'order_id': [f'ORD{i:04d}' for i in range(1, 201)],
    'customer_id': np.random.choice(range(1001, 1051), 200),
    'product_id': np.random.choice(range(1, 21), 200),
    'order_date': np.random.choice(dates, 200),
    'quantity': np.random.randint(1, 6, 200),
    'unit_price': np.random.choice([29000, 39000, 49000, 59000, 79000, 99000], 200)
}

orders_df = pd.DataFrame(orders_data)
orders_df['total_amount'] = orders_df['quantity'] * orders_df['unit_price']
orders_df.to_csv('orders.csv', index=False, encoding='utf-8-sig')

conn = sqlite3.connect('ecommerce.db')

customers_data = {
    'customer_id': range(1001, 1051),
    'customer_name': [f'고객{i}' for i in range(1, 51)],
    'region': np.random.choice(['서울', '경기', '부산', '대구', '인천'], 50),
    'membership_level': np.random.choice(['Bronze', 'Silver', 'Gold', 'Platinum'], 50),
    'join_date': pd.date_range(start='2024-01-01', periods=50, freq='W')
}
customers_df = pd.DataFrame(customers_data)
customers_df.to_sql('customers', conn, if_exists='replace', index=False)

products_data = {
    'product_id': range(1, 21),
    'product_name': [f'상품{chr(65+i)}' for i in range(20)],
    'category': np.random.choice(['전자제품', '의류', '식품', '도서'], 20),
    'brand': np.random.choice(['브랜드A', '브랜드B', '브랜드C', '브랜드D'], 20),
    'stock_quantity': np.random.randint(10, 500, 20)
}
products_df = pd.DataFrame(products_data)
products_df.to_sql('products', conn, if_exists='replace', index=False)
conn.close()

# 데이터 로드
orders = pd.read_csv('orders.csv', parse_dates=['order_date'])
conn = sqlite3.connect('ecommerce.db')
customers = pd.read_sql('SELECT * FROM customers', conn)
products = pd.read_sql('SELECT * FROM products', conn)
conn.close()

# 통합 데이터 생성
full_data = orders.merge(customers, on='customer_id', how='left') \
                  .merge(products, on='product_id', how='left')

print("✅ 데이터 준비 완료")

---

### Q1. CSV 파일 읽기 및 기본 탐색 ⭐

**문제**: `orders.csv` 파일을 읽고, 전체 주문 건수와 총 매출액을 출력하세요.

In [ ]:
# Q1 정답

# 1. CSV 파일 읽기
orders_q1 = pd.read_csv('orders.csv', parse_dates=['order_date'])

# 2. 주문 건수 계산
total_orders = len(orders_q1)

# 3. 총 매출액 계산
total_sales = orders_q1['total_amount'].sum()

# 4. 결과 출력
print(f"전체 주문 건수: {total_orders}건")
print(f"총 매출액: {total_sales:,}원")

In [ ]:
# 테스트
assert total_orders == 200, "주문 건수가 200건이어야 합니다"
assert total_sales > 0, "총 매출액은 0보다 커야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `read_csv`로 CSV 파일을 DataFrame으로 로드
- `len()`으로 행 개수 계산 (주문 건수)
- `sum()`으로 컬럼 합계 계산 (총 매출액)

**핵심 개념**:
- `parse_dates=['order_date']`: 날짜 컬럼을 datetime 타입으로 자동 변환
- `sum()`: 숫자 컬럼의 합계를 빠르게 계산

**대안 솔루션**:
```python
# shape 속성 사용
total_orders = orders_q1.shape[0]
```

**흔한 실수**:
- ❌ `count()` 사용: NaN을 제외하므로 정확한 행 개수가 아닐 수 있음
- ❌ 날짜 파싱 누락: 문자열로 저장되어 날짜 계산 불가

**실무 팁**:
CSV 파일 로드 시 항상 `encoding='utf-8-sig'` 지정하여 한글 깨짐 방지

---

### Q2. SQLite 데이터 조회 ⭐

**문제**: `ecommerce.db`에서 `customers` 테이블을 읽고, 지역별 고객 수를 출력하세요.

In [ ]:
# Q2 정답

# 1. SQLite 연결
conn_q2 = sqlite3.connect('ecommerce.db')

# 2. customers 테이블 읽기
customers_q2 = pd.read_sql('SELECT * FROM customers', conn_q2)

# 3. 연결 종료
conn_q2.close()

# 4. 지역별 고객 수 계산
region_counts = customers_q2['region'].value_counts()

# 5. 결과 출력
print("지역별 고객 수:")
for region, count in region_counts.items():
    print(f"{region}      {count}명")

In [ ]:
# 테스트
assert len(customers_q2) == 50, "고객 데이터는 50명이어야 합니다"
assert region_counts.sum() == 50, "전체 고객 수는 50명이어야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `sqlite3.connect()`로 DB 연결
- `pd.read_sql()`로 SQL 쿼리 결과를 DataFrame으로 변환
- `value_counts()`로 카테고리별 빈도 계산

**핵심 개념**:
- `read_sql()`: SQL 쿼리를 실행하고 결과를 DataFrame으로 반환
- `value_counts()`: 각 값의 출현 빈도를 자동으로 계산

**대안 솔루션**:
```python
# SQL 쿼리로 집계
query = "SELECT region, COUNT(*) as count FROM customers GROUP BY region"
result = pd.read_sql(query, conn_q2)
```

**흔한 실수**:
- ❌ 연결 종료 누락: `conn.close()` 호출 필수 (리소스 낭비)
- ❌ 잘못된 테이블명: SQL은 대소문자 구분

**실무 팁**:
대용량 데이터는 SQL에서 집계 후 pandas로 시각화 (성능 최적화)

---

### Q3. 데이터 merge 기본 ⭐⭐

**문제**: `orders`와 `products`를 `product_id`로 결합하여, 주문에 제품 이름과 카테고리를 추가하세요.

In [ ]:
# Q3 정답

# 1. orders와 products를 product_id로 결합 (left join)
orders_with_products = orders.merge(
    products,
    on='product_id',
    how='left'
)

# 2. 결과 확인
print("✅ 데이터 결합 완료")
print(f"결합 전: {orders.shape}")
print(f"결합 후: {orders_with_products.shape}")
print(f"\n추가된 컬럼: {['product_name', 'category']}")
print(orders_with_products[['order_id', 'product_id', 'product_name', 'category']].head())

In [ ]:
# 테스트
assert 'product_name' in orders_with_products.columns, "product_name 컬럼이 있어야 합니다"
assert 'category' in orders_with_products.columns, "category 컬럼이 있어야 합니다"
assert len(orders_with_products) == len(orders), "행 개수가 유지되어야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `merge()`로 두 DataFrame을 공통 컬럼 기준으로 결합
- `how='left'`로 왼쪽(orders) 데이터는 모두 유지

**핵심 개념**:
- `on='product_id'`: 결합 기준 컬럼 (양쪽 DataFrame에 동일 이름 존재)
- `how='left'`: 왼쪽 테이블의 모든 행 유지 (SQL LEFT JOIN)

**대안 솔루션**:
```python
# inner join (매칭되는 것만)
orders_with_products = orders.merge(products, on='product_id', how='inner')
```

**흔한 실수**:
- ❌ `how='inner'` 사용: 매칭 안 되는 주문 데이터 손실
- ❌ `on` 누락: 모든 컬럼 조합으로 카테시안 곱 발생

**실무 팁**:
트랜잭션(주문) 데이터는 항상 보존하므로 `left join` 권장

---

### Q4. groupby 단일 집계 ⭐⭐

**문제**: `full_data`에서 제품 카테고리별 평균 주문 금액을 계산하세요.

In [ ]:
# Q4 정답

# 1. 카테고리별 평균 주문 금액 계산
category_avg = full_data.groupby('category')['total_amount'].mean()

# 2. 결과 정렬 및 포맷팅
category_avg = category_avg.sort_values(ascending=False)

# 3. 출력
print("카테고리별 평균 주문 금액:")
for cat, avg in category_avg.items():
    print(f"{cat:8s}  {avg:,.0f}원")

In [ ]:
# 테스트
assert len(category_avg) > 0, "집계 결과가 있어야 합니다"
assert category_avg.min() > 0, "평균 금액은 0보다 커야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `groupby()`로 카테고리별 그룹 생성
- `mean()`으로 각 그룹의 평균 계산
- `sort_values()`로 내림차순 정렬

**핵심 개념**:
- `groupby('컬럼')`: 컬럼 값이 같은 행끼리 그룹화
- `['total_amount']`: 집계 대상 컬럼 지정
- `.mean()`: 각 그룹의 평균 자동 계산

**대안 솔루션**:
```python
# agg 사용
category_avg = full_data.groupby('category')['total_amount'].agg('mean')
```

**흔한 실수**:
- ❌ `sum()` 사용: 평균이 아닌 합계 계산
- ❌ 컬럼 선택 누락: 모든 숫자 컬럼의 평균 계산 (불필요)

**실무 팁**:
평균 주문 금액(객단가)은 마케팅 전략 수립의 핵심 지표

---

### Q5. groupby 다중 집계 ⭐⭐⭐

**문제**: 등급별로 총 매출, 평균 주문 금액, 주문 건수를 한 번에 계산하세요.

In [ ]:
# Q5 정답

# 1. 등급별 다중 집계 (agg 사용)
membership_stats = full_data.groupby('membership_level')['total_amount'].agg([
    ('총매출', 'sum'),
    ('평균주문금액', 'mean'),
    ('주문건수', 'count')
]).round(0)

# 2. 총매출 기준 내림차순 정렬
membership_stats = membership_stats.sort_values('총매출', ascending=False)

# 3. 출력
print("등급별 통계:")
print(membership_stats)

In [ ]:
# 테스트
assert '총매출' in membership_stats.columns, "총매출 컬럼이 있어야 합니다"
assert '평균주문금액' in membership_stats.columns, "평균주문금액 컬럼이 있어야 합니다"
assert '주문건수' in membership_stats.columns, "주문건수 컬럼이 있어야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `agg()`로 여러 집계 함수를 한 번에 적용
- 튜플 형식으로 (새 컬럼명, 집계 함수) 지정
- `round(0)`로 소수점 제거

**핵심 개념**:
- `agg([...])`: 리스트로 여러 집계 함수 동시 적용
- 튜플 (`('새이름', '함수')`): 컬럼명 커스터마이징
- `sum`, `mean`, `count`: 가장 많이 쓰는 3가지 집계

**대안 솔루션**:
```python
# 딕셔너리 형식
membership_stats = full_data.groupby('membership_level').agg({
    'total_amount': ['sum', 'mean', 'count']
})
```

**흔한 실수**:
- ❌ 개별 집계 후 concat: 비효율적이고 코드 길어짐
- ❌ 컬럼명 미지정: 'sum', 'mean' 등 자동 생성 (가독성 낮음)

**실무 팁**:
총매출(규모), 평균금액(객단가), 건수(트래픽)는 비즈니스 분석의 핵심 3요소

---

### Q6. 다차원 groupby ⭐⭐⭐

**문제**: 지역과 등급별로 총 매출을 계산하고, 내림차순으로 정렬하세요.

In [ ]:
# Q6 정답

# 1. 지역 + 등급별 총 매출 집계
region_membership_sales = full_data.groupby(['region', 'membership_level'])['total_amount'].sum()

# 2. reset_index로 DataFrame 변환
result_q6 = region_membership_sales.reset_index()
result_q6.columns = ['지역', '등급', '총매출']

# 3. 총매출 기준 내림차순 정렬
result_q6 = result_q6.sort_values('총매출', ascending=False)

# 4. 출력 (상위 10개)
print("지역 + 등급별 총매출 (Top 10):")
print(result_q6.head(10))

In [ ]:
# 테스트
assert len(result_q6.columns) == 3, "3개 컬럼이어야 합니다"
assert result_q6['총매출'].iloc[0] >= result_q6['총매출'].iloc[-1], "내림차순이어야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `groupby(['컬럼1', '컬럼2'])`로 다차원 그룹화
- `reset_index()`로 MultiIndex를 일반 컬럼으로 변환
- `sort_values()`로 정렬

**핵심 개념**:
- 다차원 groupby: 여러 컬럼 조합으로 그룹화 (교차 분석)
- `reset_index()`: 인덱스를 컬럼으로 변환 (시각화/분석에 유리)

**대안 솔루션**:
```python
# as_index=False로 자동 reset
result_q6 = full_data.groupby(['region', 'membership_level'], as_index=False)['total_amount'].sum()
```

**흔한 실수**:
- ❌ MultiIndex 유지: 시각화 시 복잡함
- ❌ 컬럼 순서 무시: 중요한 컬럼을 먼저 배치

**실무 팁**:
지역×등급, 카테고리×브랜드 등 2차원 교차 분석은 세그먼트 전략 수립에 필수

---

### Q7. Plotly 막대 차트 생성 ⭐⭐⭐

**문제**: 브랜드별 총 매출을 막대 차트로 시각화하세요.

In [ ]:
# Q7 정답

# 1. 브랜드별 총 매출 집계
brand_sales = full_data.groupby('brand')['total_amount'].sum().sort_values(ascending=False)

# 2. Graph Objects로 막대 차트 생성
fig_q7 = go.Figure()

fig_q7.add_trace(go.Bar(
    x=brand_sales.index,
    y=brand_sales.values,
    text=brand_sales.values.round(0).astype(int),  # 막대 위 텍스트
    textposition='auto',
    marker_color='lightseagreen',
    name='브랜드별 매출'
))

# 3. 레이아웃 설정
fig_q7.update_layout(
    title='브랜드별 총매출',
    xaxis_title='브랜드',
    yaxis_title='매출액 (원)',
    template='plotly_white',
    height=400
)

# 4. 차트 표시
fig_q7.show()
print("✅ 브랜드별 매출 차트 생성 완료")

### 💡 풀이 설명

**접근 방법**:
- `go.Figure()`로 빈 차트 객체 생성
- `add_trace(go.Bar(...))`로 막대 차트 추가
- `update_layout()`으로 제목, 축 레이블 설정

**핵심 개념**:
- `text=`: 막대 위 데이터 레이블 표시
- `textposition='auto'`: 자동으로 최적 위치 배치
- `template='plotly_white'`: 깔끔한 화이트 테마

**대안 솔루션**:
```python
# Plotly Express 사용 (간결)
import plotly.express as px
fig = px.bar(brand_sales, x=brand_sales.index, y=brand_sales.values)
```

**흔한 실수**:
- ❌ text 포맷팅 누락: 소수점 표시로 가독성 저하
- ❌ template 미지정: 기본 테마가 어두움

**실무 팁**:
Graph Objects는 세밀한 커스터마이징에 강점, Express는 빠른 프로토타입에 유리

---

### Q8. 서브플롯 대시보드 ⭐⭐⭐⭐

**문제**: 1행 2열 레이아웃의 대시보드를 만드세요. (왼쪽: 지역별 주문건수, 오른쪽: 카테고리별 매출 비율)

In [ ]:
# Q8 정답

# 1. 데이터 준비
region_orders = full_data['region'].value_counts()
category_sales = full_data.groupby('category')['total_amount'].sum()

# 2. 서브플롯 생성 (1행 2열)
fig_q8 = make_subplots(
    rows=1, cols=2,
    subplot_titles=('지역별 주문건수', '카테고리별 매출 비율'),
    specs=[[{'type': 'bar'}, {'type': 'pie'}]]
)

# 3. 왼쪽: 지역별 주문건수 (막대)
fig_q8.add_trace(
    go.Bar(
        x=region_orders.index,
        y=region_orders.values,
        marker_color='lightsalmon',
        name='주문건수'
    ),
    row=1, col=1
)

# 4. 오른쪽: 카테고리별 매출 비율 (파이)
fig_q8.add_trace(
    go.Pie(
        labels=category_sales.index,
        values=category_sales.values,
        name='매출비율'
    ),
    row=1, col=2
)

# 5. 레이아웃 업데이트
fig_q8.update_layout(
    title_text='📊 주문 분석 대시보드',
    showlegend=False,
    height=400,
    template='plotly_white'
)

fig_q8.show()
print("✅ 대시보드 생성 완료")

### 💡 풀이 설명

**접근 방법**:
- `make_subplots()`로 레이아웃 정의
- `specs`로 각 위치의 차트 타입 지정
- `add_trace(..., row=, col=)`로 차트 배치

**핵심 개념**:
- `rows=1, cols=2`: 1행 2열 격자
- `specs=[[{'type': 'bar'}, {'type': 'pie'}]]`: 차트 타입 명시
- `row=1, col=1`: 차트 위치 지정

**대안 솔루션**:
```python
# 2행 1열로 변경
fig = make_subplots(rows=2, cols=1, ...)
```

**흔한 실수**:
- ❌ specs 미지정: 기본 scatter로 설정되어 파이 차트 오류
- ❌ row/col 번호 오류: 1-based indexing (0이 아님)

**실무 팁**:
1행 2열은 비교 분석에 유리, 2행 2열은 다양한 지표 종합에 적합

---

### Q9. 커스텀 hover 정보 ⭐⭐⭐⭐

**문제**: 카테고리별 매출 막대 차트에 hover 시 총매출, 주문건수, 평균 주문금액을 표시하세요.

In [ ]:
# Q9 정답

# 1. 카테고리별 상세 통계 집계
category_detail = full_data.groupby('category').agg({
    'total_amount': ['sum', 'mean', 'count']
}).round(0)

category_detail.columns = ['총매출', '평균주문금액', '주문건수']
category_detail = category_detail.reset_index()

# 2. Graph Objects 차트 생성
fig_q9 = go.Figure()

fig_q9.add_trace(go.Bar(
    x=category_detail['category'],
    y=category_detail['총매출'],
    marker_color='mediumpurple',
    # customdata에 추가 정보 저장
    customdata=category_detail[['주문건수', '평균주문금액']],
    # hovertemplate으로 hover 정보 커스터마이징
    hovertemplate=
        '<b>%{x}</b><br>' +
        '총매출: %{y:,.0f}원<br>' +
        '주문건수: %{customdata[0]:.0f}건<br>' +
        '평균 주문금액: %{customdata[1]:,.0f}원<br>' +
        '<extra></extra>'
))

# 3. 레이아웃 설정
fig_q9.update_layout(
    title='카테고리별 매출 (상세 정보)',
    xaxis_title='카테고리',
    yaxis_title='총매출 (원)',
    template='plotly_white',
    height=400,
    hovermode='closest'
)

fig_q9.show()
print("✅ 커스텀 hover 차트 생성 완료")

### 💡 풀이 설명

**접근 방법**:
- `customdata`에 추가 정보를 저장
- `hovertemplate`로 hover 형식 정의
- `%{customdata[0]}`로 추가 데이터 참조

**핵심 개념**:
- `customdata=`: DataFrame 또는 배열 형태로 추가 정보 전달
- `hovertemplate=`: HTML 스타일 템플릿 문자열
- `%{x}`, `%{y}`, `%{customdata[i]}`: 데이터 참조 문법
- `<extra></extra>`: 기본 trace 이름 제거

**대안 솔루션**:
```python
# text 파라미터 사용 (간단하지만 제한적)
text=[f"{x}건" for x in counts]
```

**흔한 실수**:
- ❌ customdata 형식 오류: 2D 배열 필요 (단일 컬럼도 [[]])
- ❌ 인덱스 오류: customdata는 0-based (`[0]`, `[1]`, ...)

**실무 팁**:
hover 정보는 3-4개 핵심 지표만 표시 (너무 많으면 가독성 저하)

---

### Q10. 통합 분석 리포트 ⭐⭐⭐⭐⭐

**문제**: 월별 매출 추이, Top 5 제품 매출, 지역별 비율, 등급별 평균 금액을 2×2 대시보드로 구성하고 HTML로 저장하세요.

In [ ]:
# Q10 정답

# 1. 데이터 준비
# 1-1. 월별 매출
full_data['year_month'] = full_data['order_date'].dt.to_period('M').astype(str)
monthly_sales = full_data.groupby('year_month')['total_amount'].sum()

# 1-2. Top 5 제품 매출
product_sales = full_data.groupby('product_name')['total_amount'].sum().nlargest(5)

# 1-3. 지역별 매출 비율
region_sales_ratio = full_data.groupby('region')['total_amount'].sum()

# 1-4. 등급별 평균 주문 금액
membership_avg = full_data.groupby('membership_level')['total_amount'].mean()

# 2. 서브플롯 생성 (2행 2열)
fig_q10 = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        '월별 매출 추이',
        'Top 5 제품 매출',
        '지역별 매출 비율',
        '등급별 평균 주문금액'
    ),
    specs=[
        [{'type': 'scatter'}, {'type': 'bar'}],
        [{'type': 'pie'}, {'type': 'bar'}]
    ]
)

# 3-1. 월별 매출 추이 (선 그래프)
fig_q10.add_trace(
    go.Scatter(
        x=monthly_sales.index,
        y=monthly_sales.values,
        mode='lines+markers',
        name='월별 매출',
        line=dict(color='blue', width=2),
        marker=dict(size=8)
    ),
    row=1, col=1
)

# 3-2. Top 5 제품 매출 (막대)
fig_q10.add_trace(
    go.Bar(
        x=product_sales.index,
        y=product_sales.values,
        marker_color='orange',
        name='제품 매출'
    ),
    row=1, col=2
)

# 3-3. 지역별 매출 비율 (파이)
fig_q10.add_trace(
    go.Pie(
        labels=region_sales_ratio.index,
        values=region_sales_ratio.values,
        name='지역 비율'
    ),
    row=2, col=1
)

# 3-4. 등급별 평균 주문금액 (막대)
fig_q10.add_trace(
    go.Bar(
        x=membership_avg.index,
        y=membership_avg.values,
        marker_color='green',
        name='평균 금액'
    ),
    row=2, col=2
)

# 4. 레이아웃 업데이트
fig_q10.update_layout(
    title_text='📊 종합 매출 분석 리포트',
    showlegend=False,
    height=800,
    template='plotly_white'
)

# 5. HTML 파일로 저장
fig_q10.write_html('comprehensive_report.html')

print("✅ 통합 리포트 생성 완료")
print("📁 파일명: comprehensive_report.html")
fig_q10.show()

### 💡 풀이 설명

**접근 방법**:
1. 4가지 분석을 개별적으로 수행
2. `make_subplots`로 2×2 레이아웃 생성
3. 각 차트를 적절한 위치에 배치
4. `write_html()`로 파일 저장

**핵심 개념**:
- 월별 집계: `dt.to_period('M')`로 연월 추출
- Top N: `nlargest(5)`로 상위 5개 선택
- 다양한 차트 타입 조합: 선/막대/파이
- HTML 저장: `write_html(파일명)`

**대안 솔루션**:
```python
# 월별 집계 (다른 방식)
full_data['month'] = full_data['order_date'].dt.month
monthly = full_data.groupby('month')['total_amount'].sum()
```

**흔한 실수**:
- ❌ Period 타입 미변환: 문자열로 변환하지 않으면 시각화 오류
- ❌ specs 불일치: 차트 타입과 위치가 맞지 않음
- ❌ 파일 경로 오류: 절대 경로 또는 상대 경로 확인

**실무 팁**:
- 대시보드는 **스토리텔링 순서**로 배치 (시간 → 제품 → 지역 → 고객)
- HTML 리포트는 **이메일 자동 발송**과 연동 가능
- 크론잡으로 매일 아침 최신 데이터로 리포트 생성 자동화
- `height=800`으로 충분한 공간 확보 (2×2는 최소 800px)

---

## 📊 학습 정리

이 실습을 통해 **다중 소스 데이터 통합**, **groupby 집계**, **Plotly 대시보드** 생성까지 
실무 데이터 분석의 전체 워크플로우를 경험했습니다!

**다음 단계**: Week 2-3 통합 프로젝트에서 이 기술들을 활용하여 실전 프로젝트를 완성하세요! 🚀